In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

from lightfm import LightFM
from lightfm.evaluation import precision_at_k, recall_at_k

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from best_rec_lib.metrics import precision_at_k, recall_at_k
from best_rec_lib.utils import prefilter_items

In [2]:
data = pd.read_csv('../data/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
data = pd.read_csv('../data/retail_train.csv')
item_features = pd.read_csv('../data/product.csv')
user_features = pd.read_csv('../data/hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

# train test split
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [4]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


# Домашнее задание

## Подбор оптимальных гиперпараметров для LightFM

- Постройте модели с помощью библиотеки LightFM, изменяя следующие параметры
  - функция потерь, регуляризация
  - количество компонент
  - отдельно постройте модели, используя только матрицу взаимодействий и матрицу взаимодействий + признаки (набор признаков может быть различным, например как на вебинаре)
  
- Посчитайте метрики (Precision@5, MAP@5) для разных наборов гиперпараметров и выберете лучший набор


In [5]:
# import lightfm.evaluation
# pr = lightfm.evaluation.precision_at_k

In [6]:
n_items_before = data_train['item_id'].nunique()

data_train = prefilter_items(data_train, 5000, item_features)

n_items_after = data_train['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 86865 to 86865


In [7]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(2)

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
test_user_item_matrix = pd.pivot_table(data_test, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

test_user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

In [9]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [10]:
user_feat = pd.DataFrame(user_item_matrix.index)
user_feat = user_feat.merge(user_features, on='user_id', how='left')
user_feat.set_index('user_id', inplace=True)

item_feat = pd.DataFrame(user_item_matrix.columns)
item_feat = item_feat.merge(item_features, on='item_id', how='left')
item_feat.set_index('item_id', inplace=True)

user_feat.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
user_id,,,,,,,
1,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
user_feat_lightfm = pd.get_dummies(user_feat, columns=user_feat.columns.tolist())
item_feat_lightfm = pd.get_dummies(item_feat, columns=item_feat.columns.tolist())

In [12]:
model = LightFM(no_components=30,
                loss='bpr', # 'warp'
                learning_rate=0.05, 
                item_alpha=0.1, user_alpha=0.1, 
                random_state=42)

model.fit((sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
          sample_weight=coo_matrix(user_item_matrix),
          user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
          item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
          epochs=15, 
          num_threads=4) 

In [13]:
user_emb = model.get_user_representations(features=csr_matrix(user_feat_lightfm.values).tocsr())

In [14]:
user_emb

(array([-56.264465,   0.      ,   0.      , ..., -59.29657 , -43.486855,
          0.      ], dtype=float32),
 array([[-0.00166036, -0.00279121, -0.00126845, ..., -0.00335788,
          0.00175814,  0.00225585],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [-0.0007522 , -0.00060896,  0.00024947, ..., -0.00104013,
          0.00110158,  0.00045517],
        [-0.011279  , -0.0285571 , -0.03677412, ..., -0.01759906,
          0.02181818,  0.00428639],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]], dtype=float32))

In [15]:
user_emb[1].shape

(2499, 30)

In [16]:
# train_precision = precision_at_k(model, sparse_user_item, 
#                                  user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
#                                  item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
#                                  k=5).mean()

# train_precision

In [17]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [18]:
test_item_ids = np.arange(len(itemids))

In [19]:
userids

array([   1,    2,    3, ..., 2498, 2499, 2500])

In [20]:
test_item_ids

array([   0,    1,    2, ..., 4998, 4999, 5000])

In [27]:
test_item_ids = np.array([1, 2, 3, 200, 1200])
recs=model.predict(user_ids=0, item_ids=test_item_ids,
                            user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                            item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                            num_threads=4)

In [22]:
predictions

array([-19.196856, -17.848064, -19.196856, ..., -23.777666, -22.767998,
       -20.91811 ], dtype=float32)

In [30]:
test_item_ids = np.arange(len(itemids))

In [31]:
# result['LightFm'] = result['user_id'].map(lambda x: get_recommendations(x, model, sparse_user_item, N=5))

# result.apply(lambda row: precision_at_k(row['LightFm'], row['actual']), axis=1).mean()

In [32]:
#predictions
np.sort(predictions)[::-1]

array([  0.      , -11.562173, -13.527412, ..., -25.591236, -25.591236,
       -26.144815], dtype=float32)

In [33]:
model = LightFM(no_components=30,
                loss='warp', # 'warp' 'bpr'
                learning_rate=0.05, 
                item_alpha=0.1, user_alpha=0.1, 
                random_state=42)

model.fit((sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
          sample_weight=coo_matrix(user_item_matrix),
          user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
          item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
          epochs=15, 
          num_threads=4) 

In [35]:
from lightfm.evaluation import precision_at_k, recall_at_k

In [36]:
train_precision = precision_at_k(model, sparse_user_item, 
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                 k=5).mean()
train_precision

0.5232493

In [38]:
%%time
no_components_list = [5, 15, 50] 
loss_list = ['bpr', 'warp']
item_alpha_list = [0.005, 0.01, 0.1] 
user_alpha_list = [0.005, 0.01, 0.1]
params = []


for components in no_components_list:
    for loss in loss_list:
        for item_alpha in item_alpha_list:
            for user_alpha in user_alpha_list:
                
                
                model = LightFM(no_components=components,
                                loss=loss,
                                learning_rate=0.05,
                                item_alpha=item_alpha, 
                                user_alpha=user_alpha,
                                random_state=42)
                
                model.fit((sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
                          sample_weight=coo_matrix(user_item_matrix),  
                          user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                          item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                          epochs=15, num_threads=4)
                precision = precision_at_k(model, sparse_user_item, 
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                 k=5).mean()
                
                params_row = []
                params_row.append([components, loss, item_alpha, user_alpha, precision])
                params.append(params_row)               

CPU times: user 1h 2min 40s, sys: 20 s, total: 1h 3min
Wall time: 21min 49s


In [40]:
for i in sorted(params, key=lambda x: x[0][4], reverse=True):
    
    print(f'comp: {i[0][0]}, loss: {i[0][1]}, item: {i[0][2]}, user: {i[0][3]}, precision: {i[0][4]}')


comp: 50, loss: warp, item: 0.005, user: 0.005, precision: 0.534613847732544
comp: 5, loss: warp, item: 0.005, user: 0.005, precision: 0.5345338582992554
comp: 50, loss: warp, item: 0.01, user: 0.1, precision: 0.5336534976959229
comp: 5, loss: warp, item: 0.005, user: 0.1, precision: 0.5335735082626343
comp: 5, loss: warp, item: 0.1, user: 0.1, precision: 0.5335735082626343
comp: 50, loss: warp, item: 0.01, user: 0.01, precision: 0.5335735082626343
comp: 50, loss: warp, item: 0.1, user: 0.1, precision: 0.5335735082626343
comp: 5, loss: warp, item: 0.01, user: 0.1, precision: 0.5334934592247009
comp: 15, loss: warp, item: 0.005, user: 0.1, precision: 0.5334934592247009
comp: 15, loss: warp, item: 0.01, user: 0.1, precision: 0.5330132246017456
comp: 50, loss: warp, item: 0.005, user: 0.1, precision: 0.5330132246017456
comp: 50, loss: warp, item: 0.1, user: 0.01, precision: 0.532693088054657
comp: 5, loss: warp, item: 0.005, user: 0.01, precision: 0.5314926505088806
comp: 5, loss: warp, i